In [8]:
# Download the corresponding PyTorch Geometric module
%%capture
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.1+cu113
"""
%env TORCH=2.1.0+cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
import torch.nn.functional as F

In [ ]:
# # Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


# Check if CUDA is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data to the device (GPU if available)
data = data.to(device)

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # First GCN layer with ReLU activation and dropout
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        # Second GCN layer
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

    def get_ebd(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x


In [15]:
# Training the model
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Get the node features (data.x) and edge indices (data.edge_index)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # if epoch % 20 == 0:
        #     print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluate the model
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
        return acc.item()

def reset_weights(m):
    if isinstance(m, (torch.nn.Linear, GCNConv)):
        m.reset_parameters()


In [ ]:
# Model hyperparameters
in_channels = dataset.num_node_features  # 1433 (Cora input feature size)
hidden_channels = 64
out_channels = dataset.num_classes  # 7 (Cora has 7 classes)

# Initialize the model
model = GCN(in_channels, hidden_channels, out_channels).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [9]:
# Train the model
model.apply(reset_weights)
train(model, data, optimizer, criterion, epochs=50)

# Evaluate the model
acc = evaluate(model, data)
print(f'Test accuracy: {acc:.4f}')

Epoch 1/50, Loss: 1.9354
Epoch 21/50, Loss: 0.0038
Epoch 41/50, Loss: 0.0032
Test accuracy: 0.8040


In [10]:
from torch_geometric.utils import to_dense_adj
# Node features (X)
x = data.x  # Shape: [num_nodes, num_features]
# Adjacency matrix (A)
A = to_dense_adj(data.edge_index)[0]  # Shape: [num_nodes, num_nodes]
K = (data.x) @ (data.x).T
m = 1000

In [ ]:
from Perturbe_Algs import Attacks
from copy import deepcopy
# The two algorithms roughtly take 10 minutes each to run on a single GPU(A100).
attack_instance = Attacks(A,K,m, alpha = 50, filter = 'adj_norm_self_loop', max_iter=250,).to(device)
pgdavg_dataset = deepcopy(data)
pgdwst_dataset = deepcopy(data)

A_pgd_avg = attack_instance.Prob_PGD().clone().detach()
A_pgd_wst = attack_instance.Wst_PGD().clone().detach()

pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()


<ipython-input-11-972411cfba65>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
<ipython-input-11-972411cfba65>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()


In [ ]:
# Save the adjacency matrix (A)
torch.save(A_pgd_avg, 'avg_adj.pt')
torch.save(A_pgd_wst, 'wst_adj.pt')
# Save the modified datasets
torch.save(pgdavg_dataset, 'cora_pgdavg_data.pt')
torch.save(pgdwst_dataset, 'cora_pgdwst_data.pt')
print("Dataset and adjacency matrix saved.")

Dataset and adjacency matrix saved successfully to Google Drive.


In [ ]:
# Load the attacked graph adjacency matrix from folder 'cora_adj_norm'
# pgdavg_dataset = torch.load('cora_pgdavg_data.pt', weights_only=False)
# pgdwst_dataset = torch.load('cora_pgdwst_data.pt', weights_only=False)

In [ ]:
num_rep = 10

list_ebd_random = []
list_ebd_pgdavg = []
list_ebd_pgdwst = []
list_acc = []
list_acc_random = []
list_acc_pgdavg = []
list_acc_pgdwst = []

for i in range(num_rep):

  A_random = attack_instance.randomAttack().clone().detach()
  random_dataset = deepcopy(data)
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()

  model.apply(reset_weights)
  train(model, data, optimizer, criterion, epochs=50)
  test_acc = evaluate(model, data)
  acc_random = evaluate(model, random_dataset)
  acc_pgdavg = evaluate(model, pgdavg_dataset)
  acc_pgdwst = evaluate(model, pgdwst_dataset)

  list_acc.append(test_acc)
  list_acc_random.append(acc_random)
  list_acc_pgdavg.append(acc_pgdavg)
  list_acc_pgdwst.append(acc_pgdwst)


  ebd = model.get_ebd(data.x, data.edge_index)
  ebd_random = model.get_ebd(random_dataset.x, random_dataset.edge_index)
  ebd_pgdavg = model.get_ebd(pgdavg_dataset.x, pgdavg_dataset.edge_index)
  ebd_pgdwst = model.get_ebd(pgdwst_dataset.x, pgdwst_dataset.edge_index)


  p_ebd_random = torch.norm(ebd - ebd_random, p='fro')
  p_ebd_pgdavg = torch.norm(ebd - ebd_pgdavg, p='fro')
  p_ebd_pgdwst = torch.norm(ebd - ebd_pgdwst, p='fro')

  list_ebd_random.append(p_ebd_random.item())
  list_ebd_pgdavg.append(p_ebd_pgdavg.item())
  list_ebd_pgdwst.append(p_ebd_pgdwst.item())

  print(f"-----Iteration {i:d}-----")
  print(f"Test Acc: {test_acc:.4f}")
  print(f"PGD-AVG Test Acc: {acc_pgdavg:.4f}")
  print(f"PGD-WST Test Acc: {acc_pgdwst:.4f}")
  print(f"Random Test Acc: {acc_random:.4f}")


<ipython-input-16-2f1362b35aee>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()


-----Iteration 0-----
Test Acc: 0.7920
PGD-AVG Test Acc: 0.7730
PGD-WST Test Acc: 0.7770
Random Test Acc: 0.7770
-----Iteration 1-----
Test Acc: 0.7950
PGD-AVG Test Acc: 0.7750
PGD-WST Test Acc: 0.7760
Random Test Acc: 0.7890
-----Iteration 2-----
Test Acc: 0.7970
PGD-AVG Test Acc: 0.7760
PGD-WST Test Acc: 0.7800
Random Test Acc: 0.7760
-----Iteration 3-----
Test Acc: 0.8000
PGD-AVG Test Acc: 0.7820
PGD-WST Test Acc: 0.7830
Random Test Acc: 0.7780
-----Iteration 4-----
Test Acc: 0.7980
PGD-AVG Test Acc: 0.7760
PGD-WST Test Acc: 0.7820
Random Test Acc: 0.7920
-----Iteration 5-----
Test Acc: 0.8060
PGD-AVG Test Acc: 0.7830
PGD-WST Test Acc: 0.7870
Random Test Acc: 0.7880
-----Iteration 6-----
Test Acc: 0.8070
PGD-AVG Test Acc: 0.7820
PGD-WST Test Acc: 0.7830
Random Test Acc: 0.7790
-----Iteration 7-----
Test Acc: 0.7990
PGD-AVG Test Acc: 0.7790
PGD-WST Test Acc: 0.7800
Random Test Acc: 0.7820
-----Iteration 8-----
Test Acc: 0.7970
PGD-AVG Test Acc: 0.7750
PGD-WST Test Acc: 0.7800
Random 

In [17]:
import numpy as np
print(f"Test: {np.mean(list_acc)*100:.2f}, std:{np.std(list_acc)*100:.2f}")
print(f"Mean of random: {np.mean(list_acc_random)*100:.2f}, std:{np.std(list_acc_random)*100:.2f}")
print(f"Mean of Wst_PGD: {np.mean(list_acc_pgdwst)*100:.2f}, std: {np.std(list_acc_pgdwst)*100:.2f}")
print(f"Mean of Prob_PGD:{np.mean(list_acc_pgdavg)*100:.2f}, std: {np.std(list_acc_pgdavg):.2f}")


Test: 79.95, std:0.46
Mean of random: 78.32, std:0.54
Mean of Wst_PGD: 78.14, std: 0.34
Mean of Prob_PGD:77.77, std: 0.00


In [19]:
print(f"Mean of random embedding norm: {np.mean(list_ebd_random):.2f}, std:{np.std(list_ebd_random):.2f}")
print(f"Mean of Wst_PGD embedding norm: {np.mean(list_ebd_pgdwst):.2f}, std: {np.std(list_ebd_pgdwst):.2f}")
print(f"Mean of Prob_PGD embedding norm:{np.mean(list_ebd_pgdavg):.2f}, std: {np.std(list_ebd_pgdavg):.2f}")

Mean of random embedding norm: 96.38, std:2.09
Mean of Wst_PGD embedding norm: 101.34, std: 2.45
Mean of Prob_PGD embedding norm:110.39, std: 1.95


In [21]:
print(f"Test: {np.mean(list_acc)*100:.2f} \pm {np.std(list_acc)*100:.2f}")
print(f"Mean of random: {np.mean(list_acc_random)*100:.2f} \pm {np.std(list_acc_random)*100:.2f}")
print(f"Mean of Wst_PGD: {np.mean(list_acc_pgdwst)*100:.2f} \pm {np.std(list_acc_pgdwst)*100:.2f}")
print(f"Mean of Prob_PGD:{np.mean(list_acc_pgdavg)*100:.2f} \pm {np.std(list_acc_pgdavg):.2f}")
print(f"Mean of random embedding norm: {np.mean(list_ebd_random):.2f} \pm {np.std(list_ebd_random):.2f}")
print(f"Mean of Wst_PGD embedding norm: {np.mean(list_ebd_pgdwst):.2f} \pm {np.std(list_ebd_pgdwst):.2f}")
print(f"Mean of Prob_PGD embedding norm:{np.mean(list_ebd_pgdavg):.2f} \pm {np.std(list_ebd_pgdavg):.2f}")

Test: 79.95 \pm 0.46
Mean of random: 78.32 \pm 0.54
Mean of Wst_PGD: 78.14 \pm 0.34
Mean of Prob_PGD:77.77 \pm 0.00
Mean of random embedding norm: 96.38 \pm 2.09
Mean of Wst_PGD embedding norm: 101.34 \pm 2.45
Mean of Prob_PGD embedding norm:110.39 \pm 1.95
